# Social Media Multi-Agent System

In this session, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [ ]:
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import tool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:913: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import openai

openai.api_key = " "

In [ ]:
import os
#from utils import get_openai_api_key

#openai_api_key = get_openai_api_key()
import os
os.environ["OPENAI_API_KEY"] = " "
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [ ]:
import os
os.environ["SERPER_API_KEY"] = " "

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I will start by gathering information on the latest trends, key players, and noteworthy news in the field of Artificial Intelligence to ensure the content is current and relevant. 

Action: Search the internet  
Action Input: {"search_query": "latest trends in Artificial Intelligence 2023"}   


Search results: Title: The state of AI in 2023: Generative AI's breakout year | McKinsey
Link: https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai-in-2023-generative-ais-breakout-year
Snippet: The latest annual McKinsey Global Survey o

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Understanding Artificial Intelligence: Current Trends and Implications

## Introduction

Artificial Intelligence (AI) is no longer a futuristic concept confined to the realm of science fiction; it has become an integral part of our daily lives, influencing numerous facets of society and business. From virtual assistants like Siri and Alexa to complex algorithms that drive decision-making across industries, AI is reshaping how we interact with technology. This post delves into the current trends in AI, exploring its implications for various sectors while acknowledging the challenges that accompany its rapid evolution.

The advent of AI has sparked a significant transformation in how businesses operate, with applications ranging from automating routine tasks to enhancing customer experiences. As we explore these trends, it is crucial to maintain a balanced perspective, recognizing both the benefits and the potential pitfalls that arise with the integration of AI into our lives.

## The Rise of Generative AI

One of the most significant trends in AI is the rise of generative AI—technologies capable of producing content ranging from text and images to music and videos. Generative AI leverages advanced models such as Generative Adversarial Networks (GANs) and transformer architectures, which have revolutionized the creative landscape. For instance, tools like ChatGPT and DALL-E have showcased the ability to create human-like text and stunning visuals, respectively, demonstrating the vast potential of AI in content creation.

However, while the capabilities of generative AI are impressive, they raise ethical concerns regarding copyright and originality. As AI-generated content becomes more prevalent, questions arise about ownership and the potential for misuse in creating misleading information or deepfakes. It is essential for policymakers and industry leaders to establish frameworks that ensure the responsible use of these technologies while fostering innovation and accountability within the creative sectors.

## Cost Efficiency and Accessibility

Another trend shaping the AI landscape is the dramatic decrease in inference costs associated with running AI models. Recent advancements have made it significantly cheaper to deploy AI solutions, enabling organizations of all sizes to leverage these technologies for various applications. For instance, a study indicated that the cost of achieving equivalent results in AI benchmarks has decreased dramatically, making it feasible for small businesses to integrate AI into their operations effectively.

This reduction in costs is particularly beneficial for sectors like healthcare, where AI can assist with diagnostics and patient care. By utilizing AI-driven analytics, healthcare providers can offer personalized treatment plans while optimizing operational efficiency. However, the challenge remains in ensuring equitable access to these technologies, especially for underserved communities that may lack the necessary infrastructure to adopt AI solutions effectively. Addressing this disparity will be crucial in maximizing the positive impact of AI advancements.

## AI in the Workplace: Transformation and Challenges

As AI continues to infiltrate workplaces, it presents both opportunities and challenges. Many organizations are using AI to automate mundane tasks, allowing employees to focus on more strategic and creative endeavors. For example, AI-driven tools can streamline supply chain management and enhance customer service, significantly boosting productivity and reducing operational costs.

However, the transition to an AI-augmented workforce is not without its pitfalls. A significant concern is the potential for job displacement, as automation may render certain roles obsolete. According to a report by the IBM Institute for Business Value, while many companies are optimistic about AI adoption, they face hurdles in scaling their IT infrastructure to support these technologies. Thus, businesses must prioritize workforce retraining and upskilling initiatives to ensure that employees can thrive in an AI-enhanced environment, fostering a culture of adaptability and continuous learning.

## Privacy and Ethical Considerations

As AI becomes increasingly integrated into our lives, privacy concerns are at the forefront of public discourse. The collection and analysis of personal data are essential for many AI applications, yet they raise critical questions about consent and data security. The recent implementation of regulations like the General Data Protection Regulation (GDPR) in Europe reflects a growing awareness of the need to protect individual privacy rights in the age of AI.

Moreover, the development of AI agents that retain memory of user interactions poses a dilemma. While personalized AI can enhance user experience, it may conflict with privacy standards that advocate for data minimization. Striking a balance between personalization and privacy will be crucial as AI technologies continue to evolve, ensuring that user trust is maintained while still delivering innovative solutions.

## Conclusion

The landscape of Artificial Intelligence is rapidly changing, with generative AI, cost efficiency, workplace transformation, and privacy concerns dominating the conversation. While the potential of AI to drive innovation and improve efficiency is immense, it is essential to navigate the ethical and practical challenges that accompany its adoption. Policymakers, industry leaders, and society must engage in an ongoing dialogue to ensure that AI serves as a tool for empowerment rather than a source of inequality or ethical dilemmas. 

As we move forward, our ability to harness the benefits of AI while mitigating its risks will determine the trajectory of this transformative technology. By fostering a collaborative approach among stakeholders, we can work towards a future where AI enhances our capabilities and enriches our lives.
```